# Most-Popular Recommender

We have some criteria to certify "popularity"

- Most Members
- Most Rated

We can try to recommend these shows (excluding ones the user has already seen) as the recommendations engine


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from collections import defaultdict

import numpy as np
import tqdm as tqdm

from utils.evaluator import Evaluator

In [3]:
data_path = "../data/copperunion"

evaluator = Evaluator(data_path, normalize_unrated=False)
cai_and_rating = [
    (canonical_anime_id, anime.rating)
    for canonical_anime_id, anime in evaluator.canonical_anime_mapping.items()
]
cai_and_membership = [
    (canonical_anime_id, anime.membership_count)
    for canonical_anime_id, anime in evaluator.canonical_anime_mapping.items()
]

normalize_unrated=False


parsing users...: 100%|██████████| 73515/73515 [00:19<00:00, 3830.09it/s]

Total Animes: 12294
Total Users: 54077


In [6]:
cai_and_rating.sort(key=lambda x: -x[1])
cai_and_membership.sort(key=lambda x: -x[1])
rating_order = np.array([cai for cai, _ in cai_and_rating])
membership_order = np.array([cai for cai, _ in cai_and_membership])

In [12]:
def recommend(evaluator: Evaluator, ranked_items, items, k):
    n_anime = evaluator.max_anime_count
    mask = np.zeros(n_anime, dtype=bool)
    mask[items] = True
    chooseable_items = ranked_items[~mask]
    return chooseable_items[:k]


print("Rating Order")
user_history, k = evaluator.start_eval_test_set()
k_recommended_shows = np.zeros((len(user_history), k), dtype=np.int32)
for i, masked_history in tqdm.tqdm(enumerate(user_history), total=len(user_history)):
    items = masked_history.nonzero()[0]
    recommended = recommend(evaluator, rating_order, items, k)
    k_recommended_shows[i] = recommended
evaluator.end_eval_test_set(k_recommended_shows)

print("Membership Order")
user_history, k = evaluator.start_eval_test_set()
k_recommended_shows = np.zeros((len(user_history), k), dtype=np.int32)
for i, masked_history in tqdm.tqdm(enumerate(user_history), total=len(user_history)):
    items = masked_history.nonzero()[0]
    recommended = recommend(evaluator, membership_order, items, k)
    k_recommended_shows[i] = recommended
evaluator.end_eval_test_set(k_recommended_shows)

Rating Order


100%|██████████| 5409/5409 [00:00<00:00, 13706.95it/s]


This model took 0.3972 seconds.
Out of an optimal score of 1.0, you scored 0.0085.
Membership Order


100%|██████████| 5409/5409 [00:00<00:00, 13693.66it/s]


This model took 0.3980 seconds.
Out of an optimal score of 1.0, you scored 0.0622.
